In [159]:
import yfinance as yf
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

In [160]:
# Set the ticket value, start date (in 'YYYY-MM-DD' format) and the data 
# interval (options '1d' or '1wk')
ticker = 'CBA.AX'
startDate = '2015-01-01'
interval = '1wk'

# Set the desired take profit and stop loss percentages
profitTargetPercentage = 10
allowedLossPercentage = 5

# Download the data for the specified yahoo finance ticker and display
df = yf.download(ticker, start=startDate, interval=interval).round(decimals=3)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-29,85.188,85.278,84.661,85.278,56.836,949439
2015-01-05,85.238,86.004,84.094,85.576,57.035,10601293
2015-01-12,85.228,85.735,82.443,83.120,55.397,13781186
2015-01-19,83.567,85.765,82.523,85.765,57.161,11574803
2015-01-26,85.765,89.515,85.765,88.849,59.215,22386567
...,...,...,...,...,...,...
2023-05-15,98.530,99.870,96.010,99.800,99.800,11052005
2023-05-22,99.690,99.980,97.695,98.160,98.160,10866451
2023-05-29,99.360,99.700,96.260,96.910,96.910,19759325


In [161]:
# Create a copy of the original dataframe and delete the Adjusted Close and Volume columns
mod_df = df.copy().drop(columns=['Adj Close', 'Volume'])

# Calculate the 20 EMA and 40 EMA for later conditional checks
mod_df['EMA20'] = mod_df.Close.ewm(span=20, adjust=False).mean()
mod_df['EMA40'] = mod_df.Close.ewm(span=40, adjust=False).mean()

# Calculate the MACD and Signal for later conditional checks
exp1 = mod_df.Close.ewm(span=3, adjust=False).mean()
exp2 = mod_df.Close.ewm(span=10, adjust=False).mean()
mod_df['MACD'] = (exp1 - exp2)
mod_df['MACD GRAD'] = mod_df.MACD.diff() 
mod_df['Signal'] = mod_df.MACD.ewm(span=16, adjust=False).mean()

# Remove any NaN values
mod_df.dropna(inplace=True)

# Display the new dataframe
mod_df

,Open,High,Low,Close,EMA20,EMA40,MACD,MACD GRAD,Signal
Date,,,,,,,,,
2015-01-05,85.238,86.004,84.094,85.576,85.306381,85.292537,0.094818,0.094818,0.011155
2015-01-12,85.228,85.735,82.443,83.120,85.098154,85.186559,-0.656467,-0.751285,-0.067389
2015-01-19,83.567,85.765,82.523,85.765,85.161663,85.214776,-0.062541,0.593926,-0.066818
2015-01-26,85.765,89.515,85.765,88.849,85.512838,85.392055,1.167387,1.229928,0.078382
2015-02-02,88.819,93.454,88.391,92.479,86.176282,85.737760,2.719413,1.552026,0.389092
...,...,...,...,...,...,...,...,...,...
2023-05-15,98.530,99.870,96.010,99.800,100.126277,100.564009,-0.070495,0.584787,-0.883553
2023-05-22,99.690,99.980,97.695,98.160,99.939013,100.446740,-0.346673,-0.276179,-0.820391
2023-05-29,99.360,99.700,96.260,96.910,99.650535,100.274216,-0.825867,-0.479194,-0.821035


In [162]:
# Create new columns for the dataframe for buy signals, sell signals, buy price, 
# sell price and a transaction mumber

# Display a buy signal with a 1 if conditions are met, otherwise display 0
mod_df['BuySignal'] = np.where((mod_df.Close > mod_df.EMA40) & (mod_df.MACD > 0) & (mod_df.MACD > mod_df.Signal), 1, 0) 
mod_df['SellSignal'] = np.where((mod_df.Close < mod_df.EMA40), 1, 0)
mod_df['BuyPrice'] = None
mod_df['SellPrice'] = None
mod_df['TransactionNumber'] = None

# Display dataframe for verifying results
mod_df

,Open,High,Low,Close,EMA20,EMA40,MACD,MACD GRAD,Signal,BuySignal,SellSignal,BuyPrice,SellPrice,TransactionNumber
Date,,,,,,,,,,,,,,
2015-01-05,85.238,86.004,84.094,85.576,85.306381,85.292537,0.094818,0.094818,0.011155,1,0,None,None,None
2015-01-12,85.228,85.735,82.443,83.120,85.098154,85.186559,-0.656467,-0.751285,-0.067389,0,1,None,None,None
2015-01-19,83.567,85.765,82.523,85.765,85.161663,85.214776,-0.062541,0.593926,-0.066818,0,0,None,None,None
2015-01-26,85.765,89.515,85.765,88.849,85.512838,85.392055,1.167387,1.229928,0.078382,1,0,None,None,None
2015-02-02,88.819,93.454,88.391,92.479,86.176282,85.737760,2.719413,1.552026,0.389092,1,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15,98.530,99.870,96.010,99.800,100.126277,100.564009,-0.070495,0.584787,-0.883553,0,1,None,None,None
2023-05-22,99.690,99.980,97.695,98.160,99.939013,100.446740,-0.346673,-0.276179,-0.820391,0,1,None,None,None
2023-05-29,99.360,99.700,96.260,96.910,99.650535,100.274216,-0.825867,-0.479194,-0.821035,0,1,None,None,None


In [168]:
# Create a copy of the modified dataframe and drop the indicator columns 
# as they are no longer needed
mod_df_with_signals = mod_df.copy().drop(columns=['EMA20', 'EMA40', 'MACD', 'MACD GRAD', 'Signal'])

# Set a bought and transaction number variable
bought = False
transaction = 0

# Loop over each row in the modified dataframe with signals
for i in range(0, len(mod_df_with_signals)):
    # Check that a position is not already held with the stock
    if not bought:
        # If a buy signal is present
        if mod_df_with_signals.BuySignal.iloc[i]:
            # Set the buy price to the close of the row and update the dataframe with the purchase price
            buyPrice = mod_df_with_signals.Close.iloc[i]
            mod_df_with_signals.BuyPrice.iloc[i] = buyPrice

            # Set the transaction number as a string with leading zeros
            mod_df_with_signals.TransactionNumber.iloc[i] = str("{:05d}".format(transaction))

            # Toggle the bought variable to true
            bought = True  

            # Set the desired take profit and stop loss prices
            TP = buyPrice * (1 + profitTargetPercentage / 100)
            SL = buyPrice * (1 - allowedLossPercentage / 100)
    
    # If a position is already held on the stock
    else:
        # If there is not a sell signal
        if not mod_df_with_signals.SellSignal.iloc[i]:
            # Check if the open, high, low or close price is equal to or greater than the take profit
            if ((mod_df_with_signals.Open.iloc[i] >= TP) | (mod_df_with_signals.High.iloc[i] >= TP) | (mod_df_with_signals.Low.iloc[i] >= TP) | (mod_df_with_signals.Close.iloc[i] >= TP)):
                # Set the sell price to the close of the row and update the dataframe with the sell price
                mod_df_with_signals.SellPrice.iloc[i] = TP
                mod_df_with_signals.TransactionNumber.iloc[i] = str("{:05d}".format(transaction))

                # Toggle the bought variable to false
                bought = False

                # Increment the transaction number
                transaction += 1
        
        # If a sell signal is present
        else:
            # Set the sell price to the stop loss and update the dataframe with the sell price
            mod_df_with_signals.SellPrice.iloc[i] = SL
            mod_df_with_signals.TransactionNumber.iloc[i] = str("{:05d}".format(transaction))

            # Toggle the bought variable to false
            bought = False

            # Increment the transaction number
            transaction += 1

# Display the dataframe
mod_df_with_signals

,Open,High,Low,Close,BuySignal,SellSignal,BuyPrice,SellPrice,TransactionNumber
Date,,,,,,,,,
2015-01-05,85.238,86.004,84.094,85.576,1,0,85.576,None,00000
2015-01-12,85.228,85.735,82.443,83.120,0,1,None,81.2972,00000
2015-01-19,83.567,85.765,82.523,85.765,0,0,None,None,None
2015-01-26,85.765,89.515,85.765,88.849,1,0,88.849,None,00001
2015-02-02,88.819,93.454,88.391,92.479,1,0,None,None,None
...,...,...,...,...,...,...,...,...,...
2023-05-15,98.530,99.870,96.010,99.800,0,1,None,None,None
2023-05-22,99.690,99.980,97.695,98.160,0,1,None,None,None
2023-05-29,99.360,99.700,96.260,96.910,0,1,None,None,None


In [172]:
# Group the dataframe with purchases and sells by the transaction number
df_grouped_transaction = mod_df_with_signals[~mod_df_with_signals['TransactionNumber'].isnull()]
df_grouped_transaction = df_grouped_transaction[['BuyPrice', 'SellPrice', 'TransactionNumber']].copy()
df_grouped_transaction.set_index('TransactionNumber', inplace=True)
df_grouped_transaction = df_grouped_transaction.groupby(level=0).max()

# Drop NaN values to handle any positions at the end that may not be sold
df_grouped_transaction.dropna(inplace=True)

# Display the first 5 rows for verification
df_grouped_transaction[:5]

,BuyPrice,SellPrice
TransactionNumber,,
00000,85.576,81.2972
00001,88.849,84.40655
00002,87.506,83.1307
00003,87.088,82.7336
00004,81.57,77.4915


In [198]:
# Create a profits dataframe and calculate the profit from each transaction pair as a percentage
df_profits = pd.DataFrame(columns=['Profit'])
df_profits['Profit'] = (df_grouped_transaction.SellPrice - df_grouped_transaction.BuyPrice) / df_grouped_transaction.BuyPrice

# Display the first 5 rows for verification
df_profits[:5]

,Profit
TransactionNumber,
00000,-0.05
00001,-0.05
00002,-0.05
00003,-0.05
00004,-0.05


In [207]:
# Function to calculate and return the winrate, average profit and the % gain
def analyse(array):
    winrate = len(array[array.Profit > 0]) / len(array) * 100
    aveProfit = array.Profit.mean() * 100
    gain = (array.Profit + 1).cumprod()[-1] if (array.Profit + 1).cumprod()[-1] > 1 else -(1 - (array.Profit + 1).cumprod()[-1])

    return winrate, aveProfit.round(decimals=2),round(gain * 100, 2)

In [232]:
# Analyse the profits data frame and save the results
results = analyse(df_profits)

# Display the results
print(f"\
        The winrate was         {results[0]}%\n\
        The average profit was  {results[1]}%\n\
        The account gain was    {results[2]}%"
        )

        The winrate was         20.0%
        The average profit was  -2.0%
        The account gain was    -53.65%
